## Aggregation of OGGM Runoff

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr

In [6]:
#All of the climate models used
modelnames_OG = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5','EC-Earth3', 
                'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8','INM-CM5-0', 
                 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']

Alpine_basins = {'SALWEEN':'2319', 'IRRAWADDY':'2310', 'GANGES':'2306','BRAHMAPUTRA':'2302','YANGTZE' : '2433'}

basins = ['SALWEEN', 'IRRAWADDY', 'GANGES', 'BRAHMAPUTRA', 'YANGTZE']

SSPs = ['ssp126','ssp245','ssp370','ssp585']

# CMCC-CM2-SR5 & TaiESM1 only hold values for ssp585––this is model list without those GCMS
modelnames_OG_trimmed = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 
                         'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8',
                           'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

#Generic filepath to navigate to Drive folder 
fpathOG1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpathOG2 = 'Lizz Research Stuff/Runoff-intercomparison/OGGM/lschuster/runs_2023.3/output/basins/'

### Loading/Processing Data

In [3]:
#Importing all runoff data, OGGM is grouped by basin
rf_ds = {}
rf_ds_monthly = {}
for basin, ID in Alpine_basins.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc') as ds1:
        ds1 = ds1.runoff.load()
    rf_ds[basin] = ds1
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc') as ds2:
        ds_monthly = ds2.runoff_monthly.load()
    rf_ds_monthly[basin] = ds_monthly

In [4]:
#Summing individual glacier runoff into basin totals and converting kg to km^3
basin_rf_OG = {}
basin_rf_monthly_OG = {}
for basin, ID in Alpine_basins.items():
    basin_rf_OG[basin] = rf_ds[basin].sum(dim = 'rgi_id') * 1e-12
    basin_rf_monthly_OG[basin] = rf_ds_monthly[basin].sum(dim = 'rgi_id') * 1e-12

In [7]:
#creating dict of GloPy format
basin_sums_OG = {}
basin_sums_monthly_OG = {}
for s, SSP in enumerate(SSPs):
    basin_sums_OG[SSP] = {}
    basin_sums_monthly_OG[SSP] = {}
    for b, basin in enumerate(basins):
        basin_sums_OG[SSP][basin] = basin_rf_OG[basin].sel(scenario = SSP)
        basin_sums_monthly_OG[SSP][basin] = basin_rf_monthly_OG[basin].sel(scenario = SSP)

In [8]:
#Removing these GCMs for ALL SSPs--doing even 585 as these two are not included...
#... in Glo or Py so not only makeds OOGM easier but maintains GCM consistency in analysis
trimmed_basin_sums_OG = {}
trimmed_basin_sums_monthly_OG = {}
for s, SSP in enumerate(SSPs):
    trimmed_basin_sums_OG[SSP] = {}
    trimmed_basin_sums_monthly_OG[SSP] = {}
    for b, basin in enumerate(basins):
        trimmed_basin_sums_OG[SSP][basin] = xr.concat([basin_sums_OG[SSP][basin][0:4], basin_sums_OG[SSP][basin][5:-1]], dim='gcm')
        trimmed_basin_sums_monthly_OG[SSP][basin] = xr.concat([basin_sums_monthly_OG[SSP][basin][0:4], basin_sums_monthly_OG[SSP][basin][5:-1]], dim='gcm')

### CSV Readout

In [9]:
## set up filename to reflect what you're writing out, possibly in a nested loop
modelnames_all = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basins = ['SALWEEN', 'IRRAWADDY', 'GANGES', 'BRAHMAPUTRA', 'YANGTZE']

In [10]:
#Turning OGGM 2d time array into 1d df
df = {}
df_list = {}
for s, SSP in enumerate(SSPs):
    df[SSP] = {}
    df_list[SSP] = {}
    for b, basin in enumerate(basins):
        df[SSP][basin] = {}
        df_list[SSP][basin] = {}
        for m, model in enumerate(modelnames_all):
            df[SSP][basin][model] = {}
            df_list[SSP][basin][model] = []
            for n in range(101):
                df[SSP][basin][model][2000+n] = pd.DataFrame(trimmed_basin_sums_monthly_OG[SSP][basin]. sel(gcm = model, time = 2000+n))
                df_list[SSP][basin][model].append(df[SSP][basin][model][2000+n])
                
basin_sums_monthly_OG_df = {}
for s, SSP in enumerate(SSPs):
     basin_sums_monthly_OG_df[SSP] = {}
     for b, basin in enumerate(basins):
         basin_sums_monthly_OG_df[SSP][basin] = {}
         for m, model in enumerate(modelnames_all):
             basin_sums_monthly_OG_df[SSP][basin][model] = pd.concat(df_list[SSP][basin][model], axis = 0)

In [11]:
# Create new index using pandas date_range function
start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2100, 12, 1)
indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

#Creating dataframes of SSP, basin, and GCM containing all 3 global glacier models
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for b, basin in enumerate(basins):
        out_df[SSP][basin] = {}
        for m, model in enumerate(modelnames_all):
            
            oggm_values = basin_sums_monthly_OG_df[SSP][basin][model].values.flatten()
            
            out_df[SSP][basin][model] = pd.DataFrame(
                {
                    'OGGM': oggm_values,
                },
                index=indices
            )

In [12]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Load Separate/RGI 15/OGGM/'

for SSP in out_df:
    for basin in out_df[SSP]:
        for GCM in out_df[SSP][basin]:
            fname = f"runoff_{GCM}_{SSP}_{basin}.csv"

            # Define the full path of the output file
            output_path = os.path.join(output_dir, fname)

            # Save the DataFrame as CSV
            out_df[SSP][basin][GCM].to_csv(output_path, header=True, index=True)